In [1]:
import sys
import os
import enum 
import asyncio
###
from datetime import datetime
from math import floor, ceil
import time
from time import sleep
###
import json
###
from dotenv import load_dotenv
load_dotenv()
###
import zmq
zContext = zmq.asyncio.Context()
###
import logging
logger = logging.getLogger()
# from zmq.log.handlers import PUBHandler
# zmq_log_handler = PUBHandler('tcp://127.0.0.1:55356')
# zmq_log_handler.setFormatter(logging.Formatter(fmt='{name} > {message}', style='{'))
# zmq_log_handler.setFormatter(logging.Formatter(fmt='{name} #{lineno:>3} > {message}', style='{'), logging.DEBUG)
# zmq_log_handler.setRootTopic('greeter')
# logger.addHandler(zmq_log_handler)

logger.setLevel(logging.DEBUG)
###
from pymongo import MongoClient
from bson.objectid import ObjectId
import uuid
client = MongoClient()
db=client.fsmbot
###
import ccxt
exchange = getattr(ccxt, 'binance')({'apiKey': os.getenv('BINANCE_API_KEY'), 'secret': os.getenv('BINANCE_API_SECRET')})
###
from trade_machine import TradeModel, trades, load_trades

price_ticker = {}

import zmq
import zmq.asyncio
zContext = zmq.asyncio.Context()

In [2]:
async def run_ticker():
    socket = zContext.socket(zmq.SUB)
    socket.connect("tcp://localhost:5556")
    socket.setsockopt_string(zmq.SUBSCRIBE, 'TICKER')

    while True:
        tickerString = await socket.recv_string()
        _, symbol, ticker = tickerString.split()
        ticker = json.loads(ticker)
        price_ticker[symbol] = ticker
        if symbol in trades:        
            for model in trades[symbol].values():
                if model.state in ['LIVE', 'LONG']:
                    model.tick(ticker=ticker)

from threading import Thread

ticker_loop = asyncio.new_event_loop()

def setup_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

ticker_thread = Thread(target=setup_loop, args=(ticker_loop,))
ticker_thread.start()
ticker_future = asyncio.run_coroutine_threadsafe(run_ticker(), ticker_loop)  

In [8]:
trade = TradeModel(exchange, db.trades)
trade.insert(data={"state": 'DISABLED', "symbol" : 'ALGOUSDT', "trigger": 2, "stop": 1.7, "target": 1.91, "qty": 15})
trade.start()
trade.watch()
# trade.movestop(data={'$set': {'stop': 1.9}})

INFO:transitions.core:Executed callback 'insertrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Finished processing state DISABLED exit callbacks.
INFO:transitions.core:Finished processing state LIVE enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'buy_on_exchange'
INFO:transitions.core:Executed callback 'loadrec'
INFO:transitions.core:Executed callback 'set_stop_on_exchange'
INFO:transitions.core:Finished processing state LIVE exit callbacks.
INFO:transitions.core:Finished processing state LONG enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


load_trades_from_tb()

In [10]:
trade.update(data={'$set': {"target": 1.89, 'state': 'LONG'}})
# logger.setLevel(logging.INFO)

                    

INFO:transitions.core:Executed callback 'saverec'
INFO:transitions.core:Executed callback 'loadrec'
INFO:transitions.core:Executed callback 'machine_state_changed'


True

INFO:transitions.core:Executed callback 'remove_stop_on_exchange'
INFO:transitions.core:Executed callback 'sell_on_exchange'
INFO:transitions.core:Finished processing state LONG exit callbacks.
INFO:transitions.core:Finished processing state SOLD enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


In [15]:
# (capital, risk) = 50000, 0.01
# symbol, state = 'ALGOUSDT', 'DISABLED'
# (trigger, stop, target) = 1.764, 1.706, 1.923
# qty = (capital * risk) / (trigger - stop) 
# size = qty * trigger

# # trade = createTrade(symbol, state, qty, trigger, stop, target)
# # (tradeModel, tradeMachine) = add_trade_machine(trade)

# # tradeModel.start()
# # tradeModel
# qty,size

In [16]:
# (capital, risk) = 50000, 0.01
# symbol, state = 'ALGOUSDT', 'DISABLED'
# (trigger, stop, target) = 1.7531, 1.7715, 1.9914
# qty = (capital * risk) / (trigger - stop) 
# size = qty * trigger

# trade = createTrade(symbol, state, qty, trigger, stop, target)
# (tradeModel, tradeMachine) = add_trade_machine(trade)

# tradeModel.start()
# tradeModel

In [18]:
# # trades
# for symbol, trade_chunk_arr in trades.items():
#     for chunk in trade_chunk_arr:
#         model = chunk['model'] 
#         machine = chunk['machine']
#         if model.state != 'LONG':
#             continue
#         trade = model.trade
#         stop_order = exchange.fetchOrder(trade['curStopResp']['id'], symbol=symbol)
#         if exchange.fetchOrderStatus(trade['curStopResp']['id'], symbol='ALGOUSDT') == 'closed':
#             print(trade['id'], 'closed')
#         # print(chunk)
#     # print(tradeChunk)
# # tradeModel.trade